# Transformer Encoder from scratch

> This project aims to build a transformer encoder archetecture using the Transformer model paper. We will use the following diagram to design the architecture:

<style>
figure {
    display: block;
    margin-left: auto;
    margin-right: auto;
    width: 50%;
}
figcaption {
    text-align: center;
}
</style>
<figure>     
    <img src="images/The-Transformer-encoder-diagram.jpg" >
    <figcaption> Fig. 1: The Transformer encoder architecture. </figcaption>
</figure>


In [ ]:
import torch
import math
from torch import nn
from torch.nn import functional as F

## Single-head Attention
> The key concept we are interested in captioning for the transformer is attention. This is allows for the transformer to attend to different parts of another sequence when making predictions. For large language models (LLM), it is this very important aspect that is crucial for effective performance with language tasks such as sentiment analysis and text summarisation.

> We will start by building single-headed attention using the scaled dot product and then later utilise this for multi-headed attention:

<style>
figure {
    display: block;
    margin-left: auto;
    margin-right: auto;
    width: 50%;
}
figcaption {
    text-align: center;
}
</style>
<figure>     
    <img src="images/single-head-attention.jpeg" >
    <figcaption> Fig. 2: Single-head attention architecture. </figcaption>
</figure>


In [ ]:
def scaled_dot_product_attention(q, k, v, mask=None):
    """
    Calculate the single attention head values and attention.
    q, k, v must have matching leading dimensions.
    k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
    The mask has different shapes depending on its type(padding or look ahead)
    but it must be broadcastable for addition."""

    d_k = q.size()[-1]
    scaled = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(d_k)
    # print(f"scaled size: {scaled.size()}")
    if mask is not None:
        # print(f"mask size: {mask.size()}")
        scaled = scaled + mask
    attenion = F.softmax(scaled, dim=-1)
    values = torch.matmul(attenion, v)
    return values, attenion

## Multi-head Attention

> Now we can build the multi-head attention functionality. The diagram below shows how to construct multi-head attention:

<style>
figure {
    display: block;
    margin-left: auto;
    margin-right: auto;
    width: 50%;
}
figcaption {
    text-align: center;
}
</style>
<figure>     
    <img src="images/Multi-head-attention.jpeg" >
    <figcaption> Fig. 3: Multi-head attention architecture. </figcaption>
</figure>

In [ ]:
class MultiHeadAttention(nn.Module):
    
    def __init__(self, d_model, n_head):
        super().__init__()
        self.d_model = d_model
        self.n_head = n_head
        self.d_head = d_model // n_head
        self.qkv_layer = nn.Linear(d_model, 3 * d_model)
        self.linear_layer = nn.Linear(d_model, d_model)
        
    def forward(self, x, mask=None):
        batch_size, max_input_len, d_model = x.size()
        #print(f"x.size() = {x.size()}")
        qkv = self.qkv_layer(x).reshape(batch_size, max_input_len, self.n_head, 3*self.d_head).permute(0, 2, 1, 3)
        q, k, v = qkv.chunk(3, dim=-1) #splitting the last dimension into 3 parts so shape is 30x8x200x64
        #print(f"q.size(): {q.size()}, k.size(): {k.size()}, v.size(): {v.size()}")
        values, attention = scaled_dot_product_attention(q, k, v, mask=mask)
        #print(f"values.size(): {values.size()}, attention.size(): {attention.size()}")
        values = values.reshape(batch_size, max_input_len, self.n_head * self.d_head)
        #print(f"values.size(): {values.size()}")
        out = self.linear_layer(values)
        #print(f"out.size(): {out.size()}")
        return out

## Layer Normalisation

> For neural networks activation values can get very large in magnitude, causing large gradient steps to be taken when performing back propogation, causing unstable training. This can be avoided by normalising the activations of the hidden layers. There are multiple ways to normalise the activations of the hidden layers but we will opt with Z-score normalization.

> Using the layer mean and layer standard deviation and ε (to ensure non-zero division), summarised in the following formula:

<style>
figure {
    display: block;
    margin-left: auto;
    margin-right: auto;
    width: 50%;
}
figcaption {
    text-align: center;
}
</style>
<figure>     
    <img src="images/layer_norm.PNG" >
    <figcaption> Fig. 4: Layer Normalisation Formula </figcaption>
</figure>

In [ ]:
class LayerNormalisation(torch.nn.Module):
    def __init__(self, parameters_shape, eps=1e-5):
        super().__init__()
        self.parameters_shape = parameters_shape
        self.eps = eps
        self.gamma = torch.nn.Parameter(torch.ones(parameters_shape))
        self.beta = torch.nn.Parameter(torch.zeros(parameters_shape))
        
    def forward(self, x):
        dims = [-(i+1) for i in range(len(self.parameters_shape))]
        mean = x.mean(dims=dims, keepdim=True)
        #print(f"mean: {mean.size()}")
        var = ((x - mean)**2).mean(dims=dims, keepdim=True)
        std = (var + self.eps).sqrt()
        #print(f"std: {std.size()}")
        y = (x - mean) / std
        #print(f"y: {y.size()}")
        out = self.gamma * y + self.beta
        #print(f"out: {out.size()}")
        return out